# Calling Structural Variants

There are several software tools available for calling structural variants. We will use two callers in this part of the tutorial, `breakdancer` and `lumpy`

## Breakdancer

BreakDancer predicts five types of structural variants: insertions (INS), deletions (DEL), inversions (INV), inter-chromosomal translocations (CTX) and intra-chromosomal translocations (ITX) from next-generation short paired-end sequencing reads using read pairs that are mapped with unexpected separation distances or orientation.

Navigate to the `exercise2` directory:

In [ ]:
cd ../exercise2

In [ ]:
ls

We will use the Breakdancer software package to call structural variants on a yeast sample that was paired-end sequenced on the illumina HiSeq 2000. Breakdancer first needs to examine the BAM file to get information on the fragment size distribution for each sequencing library contained in the BAM file.

The breakdancer.config file has information about the sequencing library fragment size distribution. Use the `cat` command to print the contents of the `breakdancer.config` file.

__Q What is the mean and standard deviation of the fragment size?__

Run the breakdancer SV caller using the command:

In [ ]:
breakdancer-max breakdancer.config > ERR1015121.breakdancer.out

Look at the output of Breakdancer.

In [ ]:
head ERR1015121.breakdancer.out

Note that the output from Breakdancer is NOT VCF format, instead it is a simple text format with one line per SV event. 

### Exercises

1. What type of SV event is predicted at position III:83065? 

2. What is the size of this SV?

3. What is the score of this SV?

4. What type of SV event is predicted at position II:258766?

5. Convert the output of breakdancer into BED format

The BED format is explained here: [https://genome.ucsc.edu/FAQ/FAQformat.html#format1](https://genome.ucsc.edu/FAQ/FAQformat.html#format1)

To complete this task, create a command that:
1. Extracts all the deletions from the breakdancer.out file (**Hint:** use grep) 
2. Prints columns: 1, 2, 5, 7, and 9 to create a BED file with columns: chromosome, start, end, name, and score. (**Hint:** use awk to do this, e.g. `awk '{print $1"\t"$2}'`)
3. Print the resulting bed output into a file called: breakdancer.dels.bed

## Inspecting SVs with IGV

Now we will open the IGV genome browser and inspect some of the predicted structural variants. 

To do this type:

In [ ]:
igv

Open the reference genome. **Go to ' _Genomes -> Load Genome From Server…_ ' and select "S. cerevisiae (SacCer3)".** 

Load the BAM file. **Go to ' _File -> Load from File…_ '. Select the "ERR1015121.bam" BAM file and click ' _Open_ '.**

Load the BED file for the deletion calls that you created in the exercise 5 above. **Go to ' _File -> Load from File…_ '. Select the "breakdancer.dels.bed" BED file and click ' _Open_ '.**

### Exercises

Using the navigation bar, go to region II:258,500-266,700.

1. Can you see the structural variant? What type of structural variant is it? (**Hint:** you may need to zoom out a little to see the full structural variant).

2. Can you see any evidence to support this SV call?

3. Can you estimate the size of the SV?

The VCF in the `exercise1` directory was produced by another structural variant caller on the same sample as this exercise. 

4. Load the `exercise1/ERR1015121.vcf` VCF into IGV also (File - Load from file, and select `ERR1015121.vcf` in the exercise 1 directory).

5. Was the structural variant at II:258766 also called by the other structural variant software (lumpy)? If so, what was the predicted size?

Using the navigation bar, go to to region II:508,064-511,840. 

6. Is there a SV deletion called in this region by either SV caller? 

7. Is there any read support for a SV deletion in this region? If so, how many read pairs could support the deletion call (**Hint:** change the IGV view to `squished` and `View as pairs` to see any inconsistently aligned read pairs).

## Lumpy

We will use the Lumpy software package to call structural variants on a yeast sample that was paired-end sequenced on the Illumina Hiseq 2000. Lumpy is designed to take BAM files that have been aligned with `BWA-mem`.

Navigate to the `exercise 3` directory:

In [ ]:
cd ../exercise3

In [ ]:
ls

Check that there is a BAM file called `ERR1015069.bam` and an index file `ERR1015069.bam.bai` in the directory. The sequence data has already been mapped with `bwa mem` and the results are stored in `ERR1015069.bam`.

The first step for running Lumpy is to extract the read pairs that are discordantly mapped (i.e. pairs that are not mapped within the expected fragment size distribution). Use Samtools to extract these reads:

In [ ]:
samtools view -bh -F 1294 ERR1015069.bam | samtools sort -O bam -T ERR1015069.temp -o ERR1015069.discordants.bam

Now index the bam file. **Hint:** use samtools index

### Exercises

1. What does the `-F` option in `samtools view` do?

2. Which BAM flags does 1294 indicate? (**Hint:** in your web browser, visit
https://broadinstitute.github.io/picard/explain-flags.html and enter 1294 to find out)

The next step for Lumpy is to extract the reads that are only split mapped (i.e. split read alignments). This is all one single command:

In [ ]:
samtools view -h ERR1015069.bam | extractSplitReads_BwaMem -i stdin | samtools view -b - | samtools sort -O bam -T ERR1015069.temp -o ERR1015069.splitters.bam

Now index the bam file. (**Hint:** use samtools index)

Finally, call structural variants using lumpy, providing it with the original BAM file and the two BAM files we prepared earlier.

In [ ]:
lumpyexpress -B ERR1015069.bam -S ERR1015069.splitters.bam -D ERR1015069.discordants.bam -o ERR1015069.vcf

### Exercises

3. What type of SV event occurs at position IV:383393? What is the length of the SV event? 

4. What type of SV event occurs at position IV:437149? What is the length of the SV event?

Congratulations, you have sucessfully called structural variants from some NGS data. Now continue to the next section of the tutorial: [Calling structural variants from long reads](long-reads.ipynb)